# Python Basic Tips

In [2]:
import os
print(os.cpu_count())

16


In [3]:
from datetime import date
# Assign Multiple Variables
a, b, c = 1, 2, 3
# This is okay to assign immutable variable
a = b = c = 4
b = 5
print(a, b, c)

# This is BAD!
a = b = c = {}
b['ff'] = 0
print(a, b, c)

# Destruct Variables
a, b, *test, d = list(range(10))
print(a, b, test, d)

# F string formate
f'Rounded to {2222222.222:_.0f}'
f'Format to {date.today():%Y-%m-%d}'

4 5 4
{'ff': 0} {'ff': 0} {'ff': 0}
0 1 9


'Format to 2021-10-22'

# Python default param bug

In [7]:
def test(a=[]):
    if a is None:
        a = []
    a.append(1)
    print(a)
test(None)
test(None)

[1]
[1]


# Convert Class to dict

In [8]:
class A():
    def __init__(self):
        self.a = 'a'
        self.b = 9

test = A()
print(test.__dict__)

{'a': 'a', 'b': 9}


In [16]:
from collections import defaultdict
import json
obj = defaultdict(lambda: None, {
    'a': 1,
    'b': 2,
    'c': {
        'd': 4
    }
})
print(obj['a'])
print(obj['ad'])
print(json.dumps(obj, indent=2))

obj = {
    'a': 1,
    'b': 2,
    'c': {
        'd': 4
    }
}
from operator import itemgetter
a, b = itemgetter('a', 'cdddd')(obj)
print(a, b)

1
None
{
  "a": 1,
  "b": 2,
  "c": {
    "d": 4
  },
  "ad": null
}


KeyError: 'cdddd'

# Python Logger
https://www.youtube.com/watch?v=rcfmITJ2E7c

logger.basicConfig(level=10, format='%(name)s - %(message)s')

## components
- Logger: application log api
    - root logger `logging.getLogger()`
    - app logger `logging.getLogger('elasticsearch').setLevel(logging.WARNING)`
- Handler: Send logs to destination
    - FileHandler
    - Stream
- Filter: Filter before Handler
- Formatter: format logs


In [20]:
import logging
def test(a):
    b = logging.getLogger()
    print(a == b)

a = logging.getLogger()
c = logging.getLogger('haha')
test(a)
test(c)

got logger from __main__
Proof of codes always ran when import


# Python Intermediate Performance Tips
---
## test itertor vs list
This shows even pass list vs itertor have big performance differences

In [22]:
import itertools
import time

big_list = list(range(1, 1000000))
big_list_pointer = iter(big_list)

def test_pass_around(obj, nested=1000):
    if nested > 0:
        test_pass_around(obj, nested=nested-1)
    else:
        return

start_time = time.time()
test_pass_around(big_list)
print(f"--- {time.time() - start_time} seconds ---")   

start_time = time.time()
test_pass_around(big_list_pointer)
print(f"--- {time.time() - start_time} seconds ---")    

--- 0.012525796890258789 seconds ---
--- 0.00045871734619140625 seconds ---


# itertor funny stuff

In [23]:
import itertools
lst = range(3)
p1 = iter(lst)
c1, c2, c3 = itertools.tee(p1, 3)
print('child 1 pointer: normal')
for x in c1:
    print(x)
print('parent pointer: nothing, because its has a child pointer ran')
for x in p1:
    print(x)
print('silbing pointer still works')
for x in c2:
    print(x)


child 1 pointer: normal
0
1
2
parent pointer: nothing, because its has a child pointer ran
silbing pointer still works
0
1
2


In [25]:

from functools import lru_cache
from timeit import timeit
@lru_cache(maxsize=10000)
def calc_len(num):
    if num <= 1:
        return 1
    if num % 2 == 1:
        return calc_len(3 * num + 1) + 1
    else:
        return calc_len(num / 2) + 1

def task(scan=1000):
    max_len = float('-inf')
    for x in range(scan):
        cur_len = calc_len(x)
        if cur_len > max_len:
            max_len = cur_len
timeit(task, number=3)

0.0031824530015001073